### Set the stage

In [1]:
import pandas as pd
from IPython.display import display

positions = pd.DataFrame(columns=['ticker', 'shares'], data=[['VTI',87], ['VXUS', 103], ['VNQ',14], ['VOO',6],['VBK', 5]])
closing = pd.DataFrame(columns=['ticker', 'price'], data=[['VTI',149.73], ['VXUS',51.12],['VNQ', 93.43], ['VOO',270.37],['VBK', 181.52]])
target = pd.DataFrame(columns=['ticker', 'target_allocation'], data=[['VTI',47], ['VXUS', 34], ['VNQ',5], ['VOO',10],['VBK', 4]])

In [2]:
from functools import reduce
my_portfolio = reduce(lambda left,right: pd.merge(left,right,on='ticker'), [positions, closing, target])

def calc_cur_allocation(portfolio):
    portfolio['market_value'] = portfolio['shares']*portfolio['price']
    total_value = portfolio.market_value.sum()
    portfolio['current_allocation'] = 100 * portfolio['market_value'] / total_value

calc_cur_allocation(my_portfolio)
display(my_portfolio)

,ticker,shares,price,target_allocation,market_value,current_allocation
0,VTI,87,149.73,47,13026.51,58.864350
1,VXUS,103,51.12,34,5265.36,23.793172
2,VNQ,14,93.43,5,1308.02,5.910697
3,VOO,6,270.37,10,1622.22,7.330507
4,VBK,5,181.52,4,907.60,4.101274


In [3]:
print(my_portfolio.to_dict('records'))

[{'ticker': 'VTI', 'shares': 87, 'price': 149.73, 'target_allocation': 47, 'market_value': 13026.509999999998, 'current_allocation': 58.864350233238476}, {'ticker': 'VXUS', 'shares': 103, 'price': 51.12, 'target_allocation': 34, 'market_value': 5265.36, 'current_allocation': 23.793172165383098}, {'ticker': 'VNQ', 'shares': 14, 'price': 93.43, 'target_allocation': 5, 'market_value': 1308.02, 'current_allocation': 5.910696525169105}, {'ticker': 'VOO', 'shares': 6, 'price': 270.37, 'target_allocation': 10, 'market_value': 1622.22, 'current_allocation': 7.330507268283227}, {'ticker': 'VBK', 'shares': 5, 'price': 181.52, 'target_allocation': 4, 'market_value': 907.6, 'current_allocation': 4.101273807926087}]


In [4]:
def rebalance_without_new_fund(portfolio):
    total = portfolio.market_value.sum()
    portfolio["target_market_value"] = total * portfolio["target_allocation"]/100
    portfolio["fund_to_transfer"] = portfolio["target_market_value"] - portfolio["market_value"]
    portfolio["delta_shares"] = round(portfolio["fund_to_transfer"] /portfolio["price"])
    portfolio["shares"] =  portfolio["shares"] + portfolio["delta_shares"] 
    
    calc_cur_allocation(portfolio)

In [5]:
rebalance_without_new_fund(my_portfolio)
rebalancing_step1 = my_portfolio.copy()
display(my_portfolio)
print("Sum transfered fund {}".format(my_portfolio.fund_to_transfer.sum()))

,ticker,shares,price,target_allocation,market_value,current_allocation,target_market_value,fund_to_transfer,delta_shares
0,VTI,69.0,149.73,47,10331.37,46.880373,10400.9637,-2625.5463,-18.0
1,VXUS,147.0,51.12,34,7514.64,34.098975,7524.1014,2258.7414,44.0
2,VNQ,12.0,93.43,5,1121.16,5.087457,1106.4855,-201.5345,-2.0
3,VOO,8.0,270.37,10,2162.96,9.814804,2212.9710,590.7510,2.0
4,VBK,5.0,181.52,4,907.60,4.118392,885.1884,-22.4116,-0.0


Sum transfered fund 1.9326762412674725e-12


In [6]:
def reblance_with_new_fund(portfolio, fund):
    portfolio["fund_allocation"] = fund * portfolio["target_allocation"] / 100
    portfolio["delta_shares"] = round(portfolio["fund_allocation"] /portfolio["price"])
    portfolio["shares"] += portfolio["delta_shares"]
    calc_cur_allocation(portfolio)

reblance_with_new_fund(my_portfolio,1000)
display(my_portfolio)

,ticker,shares,price,target_allocation,market_value,current_allocation,target_market_value,fund_to_transfer,delta_shares,fund_allocation
0,VTI,72.0,149.73,47,10780.56,46.998303,10400.9637,-2625.5463,3.0,470.0
1,VXUS,154.0,51.12,34,7872.48,34.320406,7524.1014,2258.7414,7.0,340.0
2,VNQ,13.0,93.43,5,1214.59,5.295056,1106.4855,-201.5345,1.0,50.0
3,VOO,8.0,270.37,10,2162.96,9.429515,2212.9710,590.7510,0.0,100.0
4,VBK,5.0,181.52,4,907.60,3.956720,885.1884,-22.4116,0.0,40.0


In [7]:
my_portfolio["delta_shares"] += rebalancing_step1["delta_shares"]
display(my_portfolio)

,ticker,shares,price,target_allocation,market_value,current_allocation,target_market_value,fund_to_transfer,delta_shares,fund_allocation
0,VTI,72.0,149.73,47,10780.56,46.998303,10400.9637,-2625.5463,-15.0,470.0
1,VXUS,154.0,51.12,34,7872.48,34.320406,7524.1014,2258.7414,51.0,340.0
2,VNQ,13.0,93.43,5,1214.59,5.295056,1106.4855,-201.5345,-1.0,50.0
3,VOO,8.0,270.37,10,2162.96,9.429515,2212.9710,590.7510,2.0,100.0
4,VBK,5.0,181.52,4,907.60,3.956720,885.1884,-22.4116,0.0,40.0


In [18]:
new_trades = (('VXUS', 52),)
port = my_portfolio.copy()
port['delta_shares'] = 0
for ticker, shares in new_trades:
    row = port['ticker'] == ticker
    port.loc[row, 'shares'] += shares
    port.loc[row, 'delta_shares'] = shares

calc_cur_allocation(port)
display(port)

,ticker,shares,price,target_allocation,market_value,current_allocation,target_market_value,fund_to_transfer,delta_shares,fund_allocation
0,VTI,72.0,149.73,47,10780.56,42.117436,10400.9637,-2625.5463,0,470.0
1,VXUS,206.0,51.12,34,10530.72,41.141362,7524.1014,2258.7414,52,340.0
2,VNQ,13.0,93.43,5,1214.59,4.745154,1106.4855,-201.5345,0,50.0
3,VOO,8.0,270.37,10,2162.96,8.450241,2212.9710,590.7510,0,100.0
4,VBK,5.0,181.52,4,907.60,3.545807,885.1884,-22.4116,0,40.0
